In [ ]:
# Tensorflow implementation of code by 재야의 숨은 초보
# This code was adapted from this page:
# https://hiddenbeginner.github.io/study-notes/contents/tutorials/2023-04-20_CartRacing-v2_DQN.html

In [1]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numba as nb
import cv2

In [ ]:
# Set up environment
env = gym.make('CarRacing-v2', continuous=False)
env.reset()

Implements this paper: https://arxiv.org/pdf/1312.5602.pdf 

# Pre-process

In [2]:
def crop(img):
    img = img[:84, 6:90] # CarRacing-v2-specific cropping
    # img = cv2.resize(img, dsize=(84, 84)) # or you can simply use rescaling
    
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) / 255.0
    return img

@nb.njit(fastmath=True)
def rgb_to_grey(img):
    """
    Convert an RGB image to greyscale using the weighted method.
    """
    num_rows, num_cols, _ = img.shape
    grey_img = np.empty((num_rows, num_cols), dtype=np.uint8)
    for i, row in enumerate(img):
        for j, rgb_pixel in enumerate(row):
            # Compute weighted sum of RGB channels
            grey_img[i, j] = 0.2989 * rgb_pixel[0] + 0.5870 * rgb_pixel[1] + 0.1140 * rgb_pixel[2]

    return grey_img

In [3]:
class ImageEnv(gym.Wrapper):
    def __init__(
        self,
        env,
        skip_frames=4,
        stack_frames=4,
        initial_no_op=50,
        **kwargs
    ):
        super(ImageEnv, self).__init__(env, **kwargs)
        self.initial_no_op = initial_no_op
        self.skip_frames = skip_frames
        self.stack_frames = stack_frames

    
    def reset(self):
        # reset the original environment
        s, info = self.env.reset()

        # Do nothing for the next `self.initial_no_op` steps
        for i in range(self.initial_no_op):
            s, r, terminated, truncated, info = self.env.step(0)

        # crop image
        s = crop(s)
        s = rgb_to_grey(s)
        

        # initial observation is simply a copy of the frame 's'
        self.stacked_state = np.tile(s, (self.stack_frames, 1, 1))
        return self.stacked_state, info
    
    def step(self, action):
        # Take an action for self.skip_frames steps
        reward = 0
        for _ in range(self.skip_frames):
            s, r, terminated, truncated, info = self.env.step(action)
            reward += r
            if terminated or truncated:
                break

        s = crop(s)
        s = rgb_to_grey(s)

        # push the current frame 's' at the end of self.stacked_state
        self.stacked_state = np.concatenate((self.stacked_state[1:], s[np.newaxis]), axis=0)

        return self.stacked_state, reward, terminated, truncated, info

In [ ]:
env = gym.make('CarRacing-v2', continuous=False)
env = ImageEnv(env)

s, _ = env.reset()
# print(env.observation_space.shape[0])
print("The shape of an observation: ", s.shape)

# fig, axes = plt.subplots(1, 4, figsize=(20, 5))
# for i in range(4):
#     axes[i].imshow(s[i], cmap='gray')
#     axes[i].axis('off')
# plt.show()

# Creating Q-Network

In [4]:
class CNNActionValue(tf.keras.Model):
    def __init__(self, state_dim, action_dim, activation=tf.nn.relu):
        super(CNNActionValue, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(16, kernel_size=8, strides=4, input_shape=state_dim) # [N, 4, 84, 84] -> [N, 16, 20, 20]
        self.conv2 = tf.keras.layers.Conv2D(32, kernel_size=4, strides=2) # [N, 16, 20, 20] -> [N, 32, 9, 9]
        # self.conv1 = tf.keras.layers.Conv2D(16, (8, 8), strides=4, activation='relu', input_shape=state_dim)
        # self.conv2 = tf.keras.layers.Conv2D(32, (4, 4), strides=2, activation='relu')
        
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(256, activation=activation)
        self.fc2 = tf.keras.layers.Dense(action_dim)

    def call(self, x):
        x = tf.cast(x, tf.float32)
        x = tf.nn.relu(self.conv1(x))
        x = tf.nn.relu(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
#Generate some random input data
input_data = np.random.rand(32, 84, 84, 4).astype(np.float32)

input_data = tf.convert_to_tensor(input_data)

# Instantiate the model
model = CNNActionValue(state_dim=(4, 84, 84), action_dim=10)

# Test the model's output
output = model.call(input_data)
assert output.shape == (32, 10), f"Expected output shape (32, 10), but got {output.shape}"

# Test the model's differentiability
with tf.GradientTape() as tape:
    tape.watch(input_data)
    output = model.call(input_data)
grads = tape.gradient(output, input_data)
assert grads.shape == input_data.shape, f"Expected gradient shape {input_data.shape}, but got {grads.shape}"

# Update the model parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
for i in range(10):
    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        output = model.call(input_data)
        loss = tf.reduce_mean(output)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    

    print(f"Iteration {i+1}: Loss={loss.numpy():.4f}")

# Replay buffer

In [5]:
class ReplayBuffer:
    def __init__(self, state_dim, action_dim, max_size=int(1e6)):
        self.s = np.zeros((max_size, *state_dim), dtype=np.float32)
        self.a = np.zeros((max_size, *action_dim), dtype=np.int64)
        self.r = np.zeros((max_size, 1), dtype=np.float32)
        self.s_prime = np.zeros((max_size, *state_dim), dtype=np.float32)
        self.terminated = np.zeros((max_size, 1), dtype=np.float32)

        self.ptr = 0
        self.size = 0
        self.max_size = max_size

    def update(self, s, a, r, s_prime, terminated):
        self.s[self.ptr] = s
        self.a[self.ptr] = a
        self.r[self.ptr] = r
        self.s_prime[self.ptr] = s_prime
        self.terminated[self.ptr] = terminated
        
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)
        
    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, batch_size)
        return (
            tf.constant(self.s[ind], dtype=tf.float32),
            tf.constant(self.a[ind], dtype=tf.int32),
            tf.constant(self.r[ind], dtype=tf.float32),
            tf.constant(self.s_prime[ind], dtype=tf.float32),
            tf.constant(self.terminated[ind], dtype=tf.float32),
        )

# DQN Agent

In [6]:
class DQN:
    def __init__(
        self,
        state_dim,
        action_dim,
        learning_rate=0.00025,
        epsilon=1.0,
        epsilon_min=0.1,
        gamma=0.99,
        batch_size=32,
        warmup_steps=5000,
        buffer_size=int(1e5),
        target_update_interval=10000,
    ):
        self.action_dim = action_dim
        self.epsilon = epsilon
        self.gamma = gamma
        self.batch_size = batch_size
        self.warmup_steps = warmup_steps
        self.target_update_interval = target_update_interval

        self.network = CNNActionValue(state_dim, action_dim)
        self.target_network = CNNActionValue(state_dim, action_dim)
        self.target_network.set_weights(self.network.get_weights())
        self.optimizer = tf.keras.optimizers.RMSprop(learning_rate)

        self.buffer = ReplayBuffer(state_dim, (1, ), buffer_size)
        self.device = tf.device('gpu' if tf.test.is_built_with_cuda() else 'cpu')

        self.total_steps = 0
        self.epsilon_decay = (epsilon - epsilon_min) / 1e6


    @tf.function
    def act(self, x, training=True):
        self.network.trainable = training
        if training and ((np.random.rand() < self.epsilon) or (self.total_steps < self.warmup_steps)):
            a = np.random.randint(0, self.action_dim)
        else:
            x = tf.expand_dims(tf.convert_to_tensor(x, dtype=tf.float32), axis=0)
            q = self.network(x)
            a = tf.argmax(q, axis=1).numpy()[0]
        return a

    # def act(self, observation):
    #     if np.random.random() < self.epsilon:
    #         action = np.random.choice(self.action_dim)
    #     else:
    #         state = np.array([observation])
    #         actions = self.network(state)

    #         action = np.argmax(actions)

    #     return action
        
    
    @tf.function
    def learn(self):
        s, a, r, s_prime, terminated = self.buffer.sample(self.batch_size)
        s = tf.convert_to_tensor(s)
        a = tf.convert_to_tensor(a)
        r = tf.convert_to_tensor(r)
        s_prime = tf.convert_to_tensor(s_prime)
        terminated = tf.convert_to_tensor(terminated)

        with tf.GradientTape() as tape:
            next_q = self.target_network(s_prime)
            td_target = r + (1. - terminated) * self.gamma * tf.reduce_max(next_q, axis=-1, keepdims=True)
            q_values = tf.gather_nd(self.network(s), tf.stack((tf.range(a.shape[0]), a[:, 0]), axis=1))
            loss = tf.reduce_mean(tf.square(q_values - tf.stop_gradient(td_target)))

        gradients = tape.gradient(loss, self.network.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_variables))

        result = {
            'total_steps': self.total_steps,
            'value_loss': loss.numpy(),
        }
        return result
    
    def process(self, transition):
        result = {}
        self.total_steps += 1
        self.buffer.update(*transition)

        if self.total_steps > self.warmup_steps:
            result = self.learn()

        if self.total_steps % self.target_update_interval == 0:
            self.target_network.set_weights(self.network.get_weights())
        self.epsilon -= self.epsilon_decay
        return result

In [7]:
env = gym.make('CarRacing-v2', continuous=False)
env = ImageEnv(env)

max_steps = int(2e6)
eval_interval = 10000
state_dim = (4, 84, 84) 
action_dim = env.action_space.n

agent = DQN(state_dim, action_dim)

In [ ]:
def evaluate(n_evals=5):
    eval_env = gym.make('CarRacing-v2', continuous=False)
    eval_env = ImageEnv(eval_env)
    
    scores = 0
    for i in range(n_evals):
        (s, _), done, ret = eval_env.reset(), False, 0
        while not done:
            a = agent.act(s)
            s_prime, r, terminated, truncated, info = eval_env.step(a)
            s = s_prime
            ret += r
            done = terminated or truncated
        scores += ret
    return np.round(scores / n_evals, 4)

In [ ]:
from IPython.display import clear_output

history = {'Step': [], 'AvgReturn': []}

(s, _) = env.reset()
while True:
    a = agent.act(s)
    # print(a)
    s_prime, r, terminated, truncated, info = env.step(int(a))
    result = agent.process((s, a, r, s_prime, terminated))  # You can track q-losses over training from `result` variable.
    
    s = s_prime
    if terminated or truncated:
        s, _ = env.reset()
        
    if agent.total_steps % eval_interval == 0:
        ret = evaluate()
        history['Step'].append(agent.total_steps)
        history['AvgReturn'].append(ret)
        
        clear_output()
        plt.figure(figsize=(8, 5))
        plt.plot(history['Step'], history['AvgReturn'], 'r-')
        plt.xlabel('Step', fontsize=16)
        plt.ylabel('AvgReturn', fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid(axis='y')
        plt.show()
        
        agent.network.save_weights('dqn.h5')
    
    if agent.total_steps > max_steps:
        break

# Evaluation

In [8]:
eval_env = gym.make('CarRacing-v2', continuous=False, render_mode='rgb_array')
eval_env = ImageEnv(eval_env)

frames = []
scores = 0
(s, _), done, ret = eval_env.reset(), False, 0
while not done:
    frames.append(eval_env.render())
    s = s.astype(np.float32)
    a = agent.act(s)
    s_prime, r, terminated, truncated, info = eval_env.step(a)
    s = s_prime
    ret += r
    done = terminated or truncated
scores += ret

InvalidAction: you passed the invalid action `2`. The supported action_space is `Discrete(5)`

In [ ]:
def animate(imgs, video_name, _return=True):
    import cv2
    import os
    import string
    import random
    
    if video_name is None:
        video_name = ''.join(random.choice(string.ascii_letters) for i in range(18)) + '.webm'
    height, width, layers = imgs[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'VP90')
    video = cv2.VideoWriter(video_name, fourcc, 10, (width, height))
    
    for img in imgs:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        video.write(img)
    video.release()
    if _return:
        from IPython.display import Video
        return Video(video_name)

In [ ]:
animate(frames, "my_video.mp4")